In [1]:
import orjson

from sqlalchemy import create_engine, Column, String, Integer, func, event, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm
from shapely.wkt import dumps
import shapely
from shapely.wkb import loads as load_wkb
import random

In [2]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [3]:
from sqlalchemy_utils import database_exists, create_database
engine = create_engine('postgresql://postgres@localhost:5333/test')#,echo=True)

In [4]:
# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    with dbapi_connection.cursor() as cursor:
        cursor.execute('CREATE EXTENSION IF NOT EXISTS postgis;')

In [5]:
# Create a base class for our declarative mapping
Base = declarative_base()

# Define your SQLAlchemy model
class GeometryModel(Base):
    __tablename__ = 'geometries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry('POLYGON'))
    centroid = Column(Geometry('POINT'))


    @property
    def shapely_geom(self):
        return load_wkb(self.geom.desc) if self.geom else None

/tmp/ipykernel_23917/3638041328.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [6]:
# Create the table
Base.metadata.create_all(engine)

In [7]:
%%time
# -- orm approach
from sqlalchemy.orm import Session

# Getting the total number of rows
with Session(engine) as session:
    total_rows = session.query(GeometryModel).count()
print(total_rows)    

1063260
CPU times: user 4.15 ms, sys: 4.56 ms, total: 8.71 ms
Wall time: 185 ms


#### Query Spatial Intersection 

## Core

### Geom

Query Spatial Count-only Intersection 

In [65]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT count(geom) 
        FROM geometries 
        WHERE ST_Intersects(
            geom,
            ST_MakeEnvelope(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size},
                4326
            )
        )
        ''')
     ).fetchone()



#print(npolys, end= " " )

850 ms ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


id only

In [70]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT id
        FROM geometries 
        WHERE ST_Intersects(
            geom,
            ST_MakeEnvelope(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size},
                4326
            )
        )
        ''')
     ).fetchall()



#print(npolys, end= " " )

857 ms ± 60.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


return with geojson

In [75]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT ST_AsGeoJSON(geom)
        FROM geometries 
        WHERE ST_Intersects(
            geom,
            ST_MakeEnvelope(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size},
                4326
            )
        )
        ''')
     ).fetchall()



#print(npolys, end= " " )

2.73 s ± 1.01 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Centroid

Query Spatial Count-only Intersection 

In [76]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT count(geom) 
        FROM geometries 
        WHERE ST_Intersects(
            centroid,
            ST_MakeEnvelope(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size},
                4326
            )
        )
        ''')
     ).fetchone()



print(npolys, end= " " )

(131724,) (93384,) (107004,) (107856,) (109332,) (95916,) (69396,) (125940,) 236 ms ± 59.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
s="(110316,) (89460,) (102888,) (97608,) (99024,) (117720,) (84444,) (128844,)"

In [49]:
import numpy as np
s=s.replace("(","").replace(")","")
numbers = [int(num.strip()) for num in s.split(',') if num.strip()]

print(np.mean(numbers))

103788.0


id only

In [54]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT id
        FROM geometries 
        WHERE ST_Intersects(
            centroid,
            ST_MakeEnvelope(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size},
                4326
            )
        )
        ''')
     ).fetchall()



#print(npolys, end= " " )

324 ms ± 63.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


return with geojson

In [59]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT ST_AsGeoJSON(geom)
        FROM geometries 
        WHERE ST_Intersects(
            centroid,
            ST_MakeEnvelope(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size},
                4326
            )
        )
        ''')
     ).fetchall()



#print(npolys, end= " " )

2.68 s ± 572 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


TO DELETE? #Count-only Centroid

In [ ]:
# %%timeit
# #--- Core approach
# ## --- this is for counts only

# random_index = random.randint(1, total_rows)
    
# half_bbox_size= 6000
# # Step 3: Query for the specific row based on the random index
# with engine.connect() as conn:
#     random_row = conn.execute(
#         text(f'''
#         SELECT id, ST_AsGeoJSON(centroid) as centroid 
#         FROM geometries 
#         WHERE id = {random_index}
#         ''')
#     ).fetchone()
    
#     centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

#     npolys = conn.execute(
#         text(f'''
#         SELECT ST_AsGeoJSON(ST_Centroidgeom)
#         FROM geometries 
#         WHERE ST_Intersects(
#             centroid,
#             ST_MakeEnvelope(
#                 {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
#                 {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size},
#                 4326
#             )
#         )
#         ''')
#      ).fetchall()



# #print(npolys, end= " " )

## ORM

In [77]:
# Assuming you have a session
session = Session(bind=engine)

## GEOM 

count only

In [86]:
%%timeit
# Perform the query -- ORM approach
random_index = random.randint(1, total_rows)

random_row = (session.query(GeometryModel.id,func.ST_AsGeoJSON(func.ST_Centroid(GeometryModel.geom)))
            .filter(GeometryModel.id == random_index)
            .one_or_none())

centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

half_bbox_size= 6000

bounding_box_polygons = (
    session.query(
        func.count(GeometryModel.id))
    .filter(
        func.ST_Intersects(
            GeometryModel.geom,
            func.ST_MakeEnvelope(
                centroid_x - half_bbox_size,
                centroid_y - half_bbox_size,
                centroid_x + half_bbox_size,
                centroid_y + half_bbox_size,
                4326  # SRID, adjust if needed
            )
        )
    )
    .all()
)

#print(len(bounding_box_polygons), end= " " )

723 ms ± 189 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


id only

In [93]:
%%timeit
# Perform the query -- ORM approach
random_index = random.randint(1, total_rows)

random_row = (session.query(GeometryModel.id,func.ST_AsGeoJSON(func.ST_Centroid(GeometryModel.geom)))
            .filter(GeometryModel.id == random_index)
            .one_or_none())

centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

half_bbox_size= 6000

bounding_box_polygons = (
    session.query(
        GeometryModel.id)
    .filter(
        func.ST_Intersects(
            GeometryModel.geom,
            func.ST_MakeEnvelope(
                centroid_x - half_bbox_size,
                centroid_y - half_bbox_size,
                centroid_x + half_bbox_size,
                centroid_y + half_bbox_size,
                4326  # SRID, adjust if needed
            )
        )
    )
    .all()
)

#print(len(bounding_box_polygons), end= " " )

793 ms ± 186 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


geom as geojson

In [100]:
%%timeit
# Perform the query -- ORM approach
random_index = random.randint(1, total_rows)

random_row = (session.query(GeometryModel.id,func.ST_AsGeoJSON(func.ST_Centroid(GeometryModel.geom)))
            .filter(GeometryModel.id == random_index)
            .one_or_none())

centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

half_bbox_size= 6000

bounding_box_polygons = (
    session.query(
        func.ST_AsGeoJSON(GeometryModel.geom))
    .filter(
        func.ST_Intersects(
            GeometryModel.geom,
            func.ST_MakeEnvelope(
                centroid_x - half_bbox_size,
                centroid_y - half_bbox_size,
                centroid_x + half_bbox_size,
                centroid_y + half_bbox_size,
                4326  # SRID, adjust if needed
            )
        )
    )
    .all()
)

#print(len(bounding_box_polygons), end= " " )

3.08 s ± 518 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Centroid

count only

In [110]:
%%timeit
# Perform the query -- ORM approach
random_index = random.randint(1, total_rows)

random_row = (session.query(GeometryModel.id,func.ST_AsGeoJSON(func.ST_Centroid(GeometryModel.geom)))
            .filter(GeometryModel.id == random_index)
            .one_or_none())

centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

half_bbox_size= 6000

bounding_box_polygons = (
    session.query(
        func.count(GeometryModel.id))
    .filter(
        func.ST_Intersects(
            GeometryModel.centroid,
            func.ST_MakeEnvelope(
                centroid_x - half_bbox_size,
                centroid_y - half_bbox_size,
                centroid_x + half_bbox_size,
                centroid_y + half_bbox_size,
                4326  # SRID, adjust if needed
            )
        )
    )
    .all()
)

#print(len(bounding_box_polygons), end= " " )

255 ms ± 24.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


id only

In [114]:
%%timeit
# Perform the query -- ORM approach
random_index = random.randint(1, total_rows)

random_row = (session.query(GeometryModel.id,func.ST_AsGeoJSON(func.ST_Centroid(GeometryModel.geom)))
            .filter(GeometryModel.id == random_index)
            .one_or_none())

centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

half_bbox_size= 6000

bounding_box_polygons = (
    session.query(
        GeometryModel.id)
    .filter(
        func.ST_Intersects(
            GeometryModel.centroid,
            func.ST_MakeEnvelope(
                centroid_x - half_bbox_size,
                centroid_y - half_bbox_size,
                centroid_x + half_bbox_size,
                centroid_y + half_bbox_size,
                4326  # SRID, adjust if needed
            )
        )
    )
    .all()
)

#print(len(bounding_box_polygons), end= " " )

318 ms ± 32.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


geom as geojson

In [119]:
%%timeit
# Perform the query -- ORM approach
random_index = random.randint(1, total_rows)

random_row = (session.query(GeometryModel.id,func.ST_AsGeoJSON(func.ST_Centroid(GeometryModel.geom)))
            .filter(GeometryModel.id == random_index)
            .one_or_none())

centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

half_bbox_size= 100

bounding_box_polygons = (
    session.query(
        func.ST_AsGeoJSON(GeometryModel.geom))
    .filter(
        func.ST_Intersects(
            GeometryModel.centroid,
            func.ST_MakeEnvelope(
                centroid_x - half_bbox_size,
                centroid_y - half_bbox_size,
                centroid_x + half_bbox_size,
                centroid_y + half_bbox_size,
                4326  # SRID, adjust if needed
            )
        )
    )
    .all()
)

#print(len(bounding_box_polygons), end= " " )

6.53 ms ± 613 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
